In [3]:
from imutils import contours
import numpy as np
import argparse
import cv2


In [4]:
def cv_show(name,img):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows();

In [5]:
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

In [6]:
def sort_contours(cnts, method="left-to-right"):
	# initialize the reverse flag and sort index
	reverse = False
	i = 0
	# handle if we need to sort in reverse
	if method == "right-to-left" or method == "bottom-to-top":
		reverse = True
	# handle if we are sorting against the y-coordinate rather than
	# the x-coordinate of the bounding box
	if method == "top-to-bottom" or method == "bottom-to-top":
		i = 1
	# construct the list of bounding boxes and sort them from top to
	# bottom
	boundingBoxes = [cv2.boundingRect(c) for c in cnts]
	(cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
		key=lambda b:b[1][i], reverse=reverse))
	# return the list of sorted contours and bounding boxes
	return (cnts, boundingBoxes)

In [7]:
def order_points(pts):
	# 一共4个坐标点
	rect = np.zeros((4, 2), dtype = "float32")

	# 按顺序找到对应坐标0123分别是 左上，右上，右下，左下
	# 计算左上，右下
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]

	# 计算右上和左下
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]

	return rect

In [8]:
def transform(orig,pts):
    rect = order_points(pts);
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    # ...and now for the height of our new image
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    # take the maximum of the width and height values to reach
    # our final dimensions
    maxWidth = max(int(widthA), int(widthB))
    maxHeight = max(int(heightA), int(heightB))
    # construct our destination points which will be used to
    # map the screen to a top-down, "birds eye" view
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # calculate the perspective transform matrix and warp
    # the perspective to grab the screen
    M = cv2.getPerspectiveTransform(rect, dst)
    warp = cv2.warpPerspective(orig, M, (maxWidth, maxHeight))
    return warp

In [9]:
image = cv2.imread('test_03.png')
contours_img = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray,(5,5),0)
edged = cv2.Canny(blurred,75,200)
cv_show('edged',edged)

In [10]:
contours,hierarchy = cv2.findContours(edged,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
res = cv2.drawContours(contours_img,contours,-1,(0,0,255),3)
cv_show('contours',contours_img)

In [11]:
print(len(contours))

1


In [12]:
c = contours[0];
peri = cv2.arcLength(c,True);
approx = cv2.approxPolyDP(c, 0.02 * peri, True)


In [13]:
warped = transform(gray, approx.reshape(4, 2))
cv_show('warped',warped)

In [14]:
ret,thresh = cv2.threshold(warped,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
cv_show('thresh',thresh)

In [15]:
contours,hie = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))
tmp = thresh.copy();
cv2.drawContours(tmp,contours,-1,(0,0,255),3)
cv_show('img',tmp)

69


In [16]:
questioncnts = []
for c in contours:
    (x,y,w,h) = cv2.boundingRect(c);
    ar = w/float(h)
    
    if w>=20 and h>=20 and ar >= 0.9 and ar<=1.1:
        questioncnts.append(c)
        
questioncnts = sort_contours(questioncnts,method = "top-to-bottom")[0]
    

In [23]:
correct = 0 ;
ind = 0
for (q,i) in enumerate(np.arange(0, len(questioncnts), 5)):
    cnts = sort_contours(questioncnts[i:i+5])[0]
    bubbled = None;
    
    for (j,c) in enumerate(cnts):
        mask = np.zeros(thresh.shape,dtype = "uint8")
        cv2.drawContours(mask,[c],-1,255,-1)
        cv_show('mask',mask)
        mask = cv2.bitwise_and(thresh,thresh,mask=mask)
        total = cv2.countNonZero(mask)
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)
            
    color = (0,0,255)
    k = ANSWER_KEY[q]
    if bubbled != None and k == bubbled[1]:
        color = (0,255,0)
        correct+=1;
        
#     cv2.drawContours(warped,[cnts[k]],-1,15)
    cv2.drawContours(warped,[cnts[k]],-1,(0,0,255),3);

print(correct)
# cv_show('warped',warped);
     

3
